In [1]:
!pip install geohash-hilbert

import functools
import geohash_hilbert as ghh
import numpy as np
import pandas as pd
import unittest

from tqdm.notebook import tqdm

# Gravitational model

We use Gravitional model described in _Введение в математическое моделирование транспортных потоков_ by _Гасников А.В._

Let both $s_i$ and $d_i$ be $w_i$.

In [2]:
class GravLawModel:
    def __init__(self, coords=('x', 'y'), weight='w', gamma=0.065, n_iter=5):
        self.coords_ = coords
        self.weight_ = weight
        self.gamma_ = gamma
        self.n_iter_ = n_iter
    
    def _l2_dist_matrix(self, df):
        '''
        Calculates l2 distance matrix between all pairs of points,
        given x, y column names.
        '''
        dist_squared = 0.
        for coord in self.coords_:
            x = df[coord].values[:, np.newaxis]
            dist_squared += np.square(x - x.T)
        return np.sqrt(dist_squared)
        
    def _f_matrix(self, df):
        '''
        Calculates so-called f, a transit preference matrix.
        '''
        return np.exp(-self.gamma_ * self._l2_dist_matrix(df))
    
    def _initial_rho(self):
        '''
        Calculates inital rho matrix.
        '''
        return self.w_.reshape(1, -1) * self.w_.reshape(-1, 1) * self.f_ / (
            self.w_.reshape(1, -1) * self.f_).sum(axis=1, keepdims=True)
    
    def _regularized_rho(self, rho):
        '''
        Calculates regularized rho matrix denoted by weird letter.
        '''
        reg_rho = np.copy(rho)
        reduction_mask = rho.sum(axis=0) > self.w_
        
        if np.any(reduction_mask):
            reg_rho[:,reduction_mask] *= self.w_[np.newaxis,reduction_mask] / rho[:,reduction_mask].sum(axis=0, keepdims=True)
        return reg_rho
    
    def _updated_rho(self, reg_rho):
        '''
        Calculates updated rho matrix given regularized one.
        '''
        q = (self.w_ - reg_rho.sum(axis=1)).reshape(-1, 1)
        r = (self.w_ - reg_rho.sum(axis=0)).reshape(1, -1)
        return reg_rho + q * r * self.f_ / (self.f_ * r).sum(axis=1, keepdims=True)
    
    def compute(self, df):
        self.w_ = df[self.weight_].values
        self.f_ = self._f_matrix(df=df)
        
        rho = self._initial_rho()
        for i in range(self.n_iter_):
            next_rho = self._updated_rho(self._regularized_rho(rho))
            print(f'{i+1} MAE {np.abs(rho - next_rho).mean()}')
            rho = next_rho
        return rho

In [3]:
class TestGravModel(unittest.TestCase):
    def test_l2_dist_matrix(self):
        xy = np.array([[2.0, 5.1], [3.4, 6.3], [4.6, 7.9]])
        d = GravLawModel()._l2_dist_matrix(pd.DataFrame(xy, columns=['x', 'y']))
        
        self.assertTrue(np.allclose(d, np.array([
            [0, np.linalg.norm(xy[0] - xy[1]), np.linalg.norm(xy[0] - xy[2])],
            [np.linalg.norm(xy[1] - xy[0]), 0, np.linalg.norm(xy[1] - xy[2])],
            [np.linalg.norm(xy[2] - xy[0]), np.linalg.norm(xy[2] - xy[1]), 0],
        ])))
        
    def test_initial_rho(self):
        model = GravLawModel()
        xy = np.array([[2.0, 5.1], [3.4, 6.3], [4.6, 7.9]])
        f = model.f_ = model._f_matrix(pd.DataFrame(xy, columns=['x', 'y']))
        w = model.w_ = np.array([1, 7, 3])
        
        rho = np.array([
            [
                w[i] * w[j] * f[i, j] / sum(w[l] * f[i, l] for l in range(len(w)))
                for j in range(len(w))
            ] 
            for i in range(len(w))
        ])

        self.assertTrue(np.allclose(model._initial_rho(), rho))
    
    def test_regularized_rho(self):
        model = GravLawModel()
        xy = np.array([[2.0, 5.1], [3.4, 6.3], [4.6, 7.9]])
        f = model.f_ = model._f_matrix(pd.DataFrame(xy, columns=['x', 'y']))
        w = model.w_ = np.array([1, 7, 3])
        
        rho = model._initial_rho()
        reg_rho = np.array([
            [
                rho[i, j] * w[j] / sum(rho[l, j] for l in range(len(w)))
                if sum(rho[l, j] for l in range(len(w))) > w[j] else rho[i, j]
                for j in range(len(w))
            ]
            for i in range(len(w))
        ])

        self.assertTrue(np.allclose(model._regularized_rho(rho), reg_rho))
        
    
    def test_updated_rho(self):
        model = GravLawModel()
        xy = np.array([[2.0, 5.1], [3.4, 6.3], [4.6, 7.9]])
        f = model.f_ = model._f_matrix(pd.DataFrame(xy, columns=['x', 'y']))
        w = model.w_ = np.array([1, 7, 3])
        
        rho = model._initial_rho()
        reg_rho = model._regularized_rho(rho)
        
        q = np.array([
            w[i] - sum(reg_rho[i, j] for j in range(len(w)))
            for i in range(len(w))
        ])
        r = np.array([
            w[j] - sum(reg_rho[i, j] for i in range(len(w)))
            for j in range(len(w))
        ])
        
        updated_rho = np.array([
            [
                reg_rho[i, j] + q[i] * r[j] * f[i, j] / sum(r[l] * f[i, l] for l in range(len(w)))
                for j in range(len(w))
            ]
            for i in range(len(w))
        ])

        self.assertTrue(np.allclose(model._updated_rho(reg_rho), updated_rho))


unittest.main(argv=[''], verbosity=2, exit=False);

test_initial_rho (__main__.TestGravModel) ... ok
test_l2_dist_matrix (__main__.TestGravModel) ... ok
test_regularized_rho (__main__.TestGravModel) ... ok
test_updated_rho (__main__.TestGravModel) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.010s

OK


# Geohashing

In [4]:
def add_geohash_info(row, pbar=None):
    row['geohash'] = ghh.encode(row['lon'], row['lat'], precision=16, bits_per_char=2)
    row['geohash_lon'], row['geohash_lat'] = ghh.decode(row['geohash'], bits_per_char=2)

    if pbar is not None:
        pbar.update(1)
        
    return row

# Prepare data

In [5]:
CLUSTERS_CSV = '../data/labeled_cluster_data_2023-05-24_20-11-46.csv'

df_clusters = pd.read_csv(CLUSTERS_CSV)
df_clusters.head()

,Unnamed: 0,id,lat,lon,cluster,cluster_size,inq,wd_rate,wd_all,work_place
0,0,2.0,54.8436,38.1929,0.0,5056.0,46681.775000,1.00,1.000000,False
1,1,6.0,54.9034,38.0696,0.0,989.0,15146.533333,0.75,0.428571,True
2,2,6.0,54.8933,38.0780,1.0,830.0,39723.665414,1.00,0.571429,False
3,5,13.0,54.8428,38.1908,0.0,2908.0,38369.617647,1.00,1.000000,False
4,6,14.0,54.8419,38.1885,0.0,2213.0,35349.094340,1.00,1.000000,False


In [6]:
with tqdm(total=len(df_clusters)) as pbar:
    df_clusters_with_geohash = df_clusters.apply(functools.partial(add_geohash_info, pbar=pbar), axis=1)
df_clusters_with_geohash.head()

  0%|          | 0/4753 [00:00<?, ?it/s]

,Unnamed: 0,id,lat,lon,cluster,cluster_size,inq,wd_rate,wd_all,work_place,geohash,geohash_lon,geohash_lat
0,0,2.0,54.8436,38.1929,0.0,5056.0,46681.775000,1.00,1.000000,False,2101231011032333,38.191223,54.842377
1,1,6.0,54.9034,38.0696,0.0,989.0,15146.533333,0.75,0.428571,True,2101230300020131,38.070374,54.902802
2,2,6.0,54.8933,38.0780,1.0,830.0,39723.665414,1.00,0.571429,False,2101230300020021,38.075867,54.894562
3,5,13.0,54.8428,38.1908,0.0,2908.0,38369.617647,1.00,1.000000,False,2101231011032333,38.191223,54.842377
4,6,14.0,54.8419,38.1885,0.0,2213.0,35349.094340,1.00,1.000000,False,2101231011032333,38.191223,54.842377


In [7]:
df = df_clusters_with_geohash[
    ['geohash', 'geohash_lon', 'geohash_lat', 'cluster_size']
].groupby(
    ['geohash', 'geohash_lon', 'geohash_lat']
).agg(
    'count'
).reset_index()

print('Num rows:', len(df))
df.head()

Num rows: 377


,geohash,geohash_lon,geohash_lat,cluster_size
0,2101230300001202,37.998962,54.880829,1
1,2101230300001323,37.998962,54.867096,1
2,2101230300002002,38.020935,54.869843,1
3,2101230300002003,38.020935,54.867096,1
4,2101230300002010,38.026428,54.867096,2


# Experiments

In [8]:
rho = GravLawModel(coords=('geohash_lon', 'geohash_lat'), weight='cluster_size', n_iter=10).compute(df)

1 MAE 3.894926802883065e-05
2 MAE 2.1083889444125463e-08
3 MAE 2.9167827466520014e-12
4 MAE 4.485520307535388e-16
5 MAE 1.0919960603700652e-17
6 MAE 3.941167275657063e-18
7 MAE 5.570279463913871e-18
8 MAE 4.768794742131261e-18
9 MAE 3.3047947460575892e-18
10 MAE 2.3049083269900123e-18
